In [ ]:
import pandas as pd
df = pd.read_csv("/kaggle/input/crop-yield-prediction-dataset/yield_df.csv")
df.head()

,Unnamed: 0,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,0,Albania,Maize,1990,36613,1485.0,121.0,16.37
1,1,Albania,Potatoes,1990,66667,1485.0,121.0,16.37
2,2,Albania,"Rice, paddy",1990,23333,1485.0,121.0,16.37
3,3,Albania,Sorghum,1990,12500,1485.0,121.0,16.37
4,4,Albania,Soybeans,1990,7000,1485.0,121.0,16.37


In [ ]:
print(df.columns.tolist())


['Unnamed: 0', 'Area', 'Item', 'Year', 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']


In [ ]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
df.dtypes

Area                              object
Item                              object
Year                               int64
hg/ha_yield                        int64
average_rain_fall_mm_per_year    float64
pesticides_tonnes                float64
avg_temp                         float64
dtype: object

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np

# --- Features and target ---
X = df.drop("hg/ha_yield", axis=1)
y = df["hg/ha_yield"]

# --- Identify categorical and numerical features ---
cat_features = ["Area", "Item"]
num_features = ["average_rain_fall_mm_per_year", "pesticides_tonnes", "avg_temp", "Year"]

# --- Preprocessing: One-hot encode categorical variables ---
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
], remainder='passthrough')

# --- Split data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Random Forest Model ---
rf_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(n_estimators=200, random_state=42))
])

rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)

# --- XGBoost Model ---
xgb_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(n_estimators=300, learning_rate=0.1, random_state=42))
])

xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

# --- Evaluation ---
def evaluate(y_true, y_pred, model_name):
    print(f"\n📈 {model_name} Results:")
    print(f"R² Score: {r2_score(y_true, y_pred):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_true, y_pred)):.2f}")

evaluate(y_test, rf_preds, "Random Forest")
evaluate(y_test, xgb_preds, "XGBoost")



📈 Random Forest Results:
R² Score: 0.9877
RMSE: 9453.84

📈 XGBoost Results:
R² Score: 0.9741
RMSE: 13714.82
